In [30]:


import sys
import os
# sys.path.append('/home/beinhaud/diplomka/mcs-source/evaluation_tools')
sys.path.append('/home/david/source/diplomka/evaluation_tools')
sys.path.append('/home/david/source/diplomka/results_analysis_tools')
from typing import List, Dict, Any, Tuple, Union

from evaluation_tools.plugins.predictions_analyzer import PredictionsAnalyzer
from evaluation_tools.plugins.wandb_processor import WandbProcessor
from evaluation_tools.fields.experiment_parameters_fields import WandbExperimentVariants, GridSearchRunVariants, ModelEvaluationRunVariant, AdditionalExperiments
from evaluation_tools.fields.prediction_analysis_fields import (
    BatchSummaryFields,
    EvaluationPairsVariants,
)
from evaluation_tools.fields.dataset_analyzer_fields import AnalysisFields, DatasetVariantField
from evaluation_tools.fields.evaluation_processor_fields import EvaluationProcessorChoices
from nn_model.type_variants import EvaluationFields
import nn_model.globals

import pandas as pd
import numpy as np

from results_analysis_tools.result_analyzer import ResultAnalyzer
from evaluation_tools.fields.dataset_parameters import ALL_TIME_STEP_VARIANTS
from results_analysis_tools.fields.experiment_analyses import (
    PlottingVariants,
    PluginVariants,
)
from results_analysis_tools.results_plotter import ResultsPlotter
from results_analysis_tools.plugins.dataset_results_processor import DatasetResultsProcessor

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import interp1d

from evaluation_tools.fields.dataset_analyzer_fields import StatisticsFields



In [31]:
PLOTING_DESTINATION = "/home/david/source/diplomka/thesis_plots/"
EVALUATION_RESULTS_BASE = "/analysis_results"
analysis_paths = {
    EvaluationProcessorChoices.FULL_DATASET_ANALYSIS: f"{nn_model.globals.PROJECT_ROOT}{EVALUATION_RESULTS_BASE}/{EvaluationProcessorChoices.FULL_DATASET_ANALYSIS.value}/",
    EvaluationProcessorChoices.SUBSET_DATASET_ANALYSIS: f"{nn_model.globals.PROJECT_ROOT}{EVALUATION_RESULTS_BASE}/{EvaluationProcessorChoices.SUBSET_DATASET_ANALYSIS.value}/",
    # EvaluationProcessorChoices.WANDB_ANALYSIS: f"{nn_model.globals.PROJECT_ROOT}{EVALUATION_RESULTS_BASE}/{EvaluationProcessorChoices.WANDB_ANALYSIS.value}/",
    EvaluationProcessorChoices.PREDICTION_ANALYSIS: f"{nn_model.globals.PROJECT_ROOT}{EVALUATION_RESULTS_BASE}/{EvaluationProcessorChoices.PREDICTION_ANALYSIS.value}/",
}
result_analyzer = ResultAnalyzer(analysis_paths)
all_results = result_analyzer.get_all_results



In [32]:
def function_to_process(row: pd.Series) -> Dict[str, Any]:
    return {
        "statistics_type": row["statistics_type"],
        "values": row["values"],
    }
    

In [ ]:
separate_train_experiment = DatasetResultsProcessor.get_dataset_type(DatasetResultsProcessor.get_analysis_type(DatasetResultsProcessor.get_full_dataset_variant(all_results), AnalysisFields.TIME_BIN_SPIKE_COUNTS), is_test=False)


In [66]:
separate_train_experiment = DatasetResultsProcessor.get_dataset_type(DatasetResultsProcessor.get_analysis_type(DatasetResultsProcessor.get_full_dataset_variant(all_results), AnalysisFields.SEPARATE_EXPERIMENT_ANALYSIS), is_test=True)
data = separate_train_experiment['analysis_results'][33]
data = data[data["statistics_type"] == StatisticsFields.TOTAL_COUNT]
data['values'][0].shape

# data = DatasetResultsProcessor.reformat_dataset_dataframe(separate_train_experiment, function_to_process)
# data = data[data["statistics_type"] == StatisticsFields.SYNCHRONY]
# data["values"][24]

(2, 20)

# Time Bin Spike Counts
This part of code shows how to plot the distribution of spike counts across all time bins and how to write the exact ratios.

In [ ]:
counts = result_analyzer.get_mean_spike_counts(False)

In [ ]:
# result_analyzer = ResultAnalyzer(analysis_paths)
counts_test = result_analyzer.get_mean_spike_counts(is_test=True)

In [ ]:
is_test = False
filename = f"time_step_counts_{"test" if is_test else "train"}.pdf"
plot_data = result_analyzer.prepare_dataframe_for_plot(PlottingVariants.TIME_BIN_COUNT_RATIO, is_test=is_test)
ResultsPlotter.plot(plot_data, PlottingVariants.TIME_BIN_COUNT_RATIO, is_test=is_test, save_fig=f"{PLOTING_DESTINATION}{filename}")

In [ ]:
is_test = True
filename = f"time_step_counts_{"test" if is_test else "train"}.pdf"
plot_data = result_analyzer.prepare_dataframe_for_plot(PlottingVariants.TIME_BIN_COUNT_RATIO, is_test=is_test)
ResultsPlotter.plot(plot_data, PlottingVariants.TIME_BIN_COUNT_RATIO, is_test=is_test, save_fig=f"{PLOTING_DESTINATION}{filename}")

# Temporal Distribution of Spikes

In [ ]:
is_test = False
filename = f"temporal_spike_distribution_{"test" if is_test else "train"}.pdf"
destination = f"{PLOTING_DESTINATION}{filename}"
plot_data = result_analyzer.prepare_dataframe_for_plot(PlottingVariants.TEMPORAL_SPIKE_DISTRIBUTION, is_test=is_test)
ResultsPlotter.plot(plot_data, PlottingVariants.TEMPORAL_SPIKE_DISTRIBUTION, save_fig=destination, is_test=is_test)

In [ ]:
is_test = True
filename = f"temporal_spike_distribution_{"test" if is_test else "train"}.pdf"
destination = f"{PLOTING_DESTINATION}{filename}"
plot_data = result_analyzer.prepare_dataframe_for_plot(PlottingVariants.TEMPORAL_SPIKE_DISTRIBUTION, is_test=is_test)
ResultsPlotter.plot(plot_data, PlottingVariants.TEMPORAL_SPIKE_DISTRIBUTION, save_fig=destination, is_test=is_test)

In [ ]:
filename = "temporal_correlation_time_bin_size.pdf"
destination = f"{PLOTING_DESTINATION}{filename}"
train_corr = result_analyzer.dataset_time_binning_temporal_resolution_correlation_matrix(is_test=False)
test_corr = result_analyzer.dataset_time_binning_temporal_resolution_correlation_matrix(is_test=True)
ResultsPlotter.plot((train_corr, test_corr), PlottingVariants.CORRELATION_MATRIX_BIN_SIZE_TEMPORAL_DATASET, save_fig=destination, is_test=is_test)